In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [2]:
# trainfile = '/kaggle/input/icr-identify-age-related-conditions/train.csv'
trainfile = 'data/train.csv'
df = pd.read_csv(trainfile)
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [3]:
# testfile = '/kaggle/input/icr-identify-age-related-conditions/test.csv'
testfile = 'data/test.csv'
test_df = pd.read_csv(testfile)
test_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
non_numeric_columns = df.select_dtypes(include=['object']).columns
non_numeric_columns

Index(['Id', 'EJ'], dtype='object')

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Drop non-feature columns
df = df.drop(columns=['Id', 'EJ'])

# Split the data into features and target label
X = df.drop('Class', axis=1)
y = df['Class']

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss

from sklearn.impute import SimpleImputer

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Impute missing values in the feature set
X_imputed = imputer.fit_transform(X)

# Initialize the scaler
scaler = StandardScaler()

# Standardize the feature set
X_standardized = scaler.fit_transform(X_imputed)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y, test_size=0.2, random_state=42)

models = [
    ('Logistic Regression', LogisticRegression(random_state=42)),
    ('Support Vector Machine', SVC(probability=True, random_state=42)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),
    ('Naive Bayes', GaussianNB()),
    ('Random Forest', RandomForestClassifier(random_state=42))
]

results_standardized = []

for name, model in models:
    model.fit(X_train, y_train)
    y_test_prob = model.predict_proba(X_test)
    log_loss_score = log_loss(y_test, y_test_prob)
    results_standardized.append((name, log_loss_score))

results_df_standardized = pd.DataFrame(results_standardized, columns=['Model', 'Log Loss'])
results_df_standardized

,Model,Log Loss
0,Logistic Regression,0.276828
1,Support Vector Machine,0.274074
2,K-Nearest Neighbors,1.620857
3,Decision Tree,5.813492
4,Naive Bayes,3.610112
5,Random Forest,0.245250


In [7]:
from xgboost import XGBClassifier

# Initialize the model
model = XGBClassifier(n_estimators=500, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Make probability predictions
y_test_prob = model.predict_proba(X_test)

# Calculate the balanced logarithmic loss
log_loss_score = log_loss(y_test, y_test_prob)
log_loss_score

0.10820378456010356

In [8]:
from sklearn.ensemble import VotingClassifier

# Initialize the best models with the best hyperparameters
xgb_best_model = XGBClassifier(n_estimators=1000, eta=0.01, max_depth=5, random_state=42)
rf_best_model = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)

# Create an ensemble model
ensemble_model = VotingClassifier(estimators=[('xgb', xgb_best_model), ('rf', rf_best_model)], voting='soft')

# Train the ensemble model
ensemble_model.fit(X_train, y_train)

# Make probability predictions
y_test_prob = ensemble_model.predict_proba(X_test)

# Calculate the balanced logarithmic loss
log_loss_score = log_loss(y_test, y_test_prob)
log_loss_score

0.1757417704796444

In [9]:
test_df

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_sub = test_df.drop(columns=['Id', 'EJ']).values

In [11]:
# Impute missing values in the feature set
X_sub = imputer.fit_transform(X_sub)

# Standardize the feature set
X_sub = scaler.fit_transform(X_sub)

In [12]:
X_train.shape, X_sub.shape

((493, 55), (5, 55))

In [13]:
y_test_sub = ensemble_model.predict_proba(X_sub)

In [14]:
submission = pd.DataFrame(test_df["Id"], columns=["Id"])
submission["class_0"] = y_test_sub[:,0]
submission["class_1"] = y_test_sub[:,1]
submission.to_csv('submission.csv', index=False)

In [15]:
submission

,Id,class_0,class_1
0,00eed32682bb,0.872224,0.127776
1,010ebe33f668,0.872224,0.127776
2,02fa521e1838,0.872224,0.127776
3,040e15f562a2,0.872224,0.127776
4,046e85c7cc7f,0.872224,0.127776
